In [1]:
!python -V

Python 3.8.20


In [2]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())

11.0
True


In [ ]:
# conda install pytorch==1.7.0 torchvision==0.8.0 torchaudio==0.7.0 cudatoolkit=11.0 -c pytorch
# pip install ipywidgets --upgrade
# pip install jupyter --upgrade

In [11]:
import json

# Đường dẫn tới file JSONL
dev_path = "/home/hoang/DSI-QG/data/msmarco_data/100k/msmarco_DSI_dev_data.json"
train_path = "/home/hoang/DSI-QG/data/msmarco_data/100k/msmarco_DSI_train_data.json"

# Mở file và đọc từng dòng
with open(dev_path, 'r', encoding='utf-8') as file:
    dev_data = [json.loads(line) for line in file]
print("Number of example (all is query) in dev_data:", len(dev_data))

with open(train_path, 'r', encoding='utf-8') as file:
    train_data = [json.loads(line) for line in file]
print("Number of example in train_data:", len(train_data))
print("Number of document in train data:", (len(train_data) - len(dev_data)) // 2 + len(dev_data))
print("Number of query in train data:", (len(train_data) - len(dev_data)) // 2)

Number of example (all is query) in dev_data: 6980
Number of example in train_data: 193020
Number of document in train data: 100000
Number of query in train data: 93020


In [4]:
%cd /home/hoang/DSI-QG/data
!python process_marco.py --train_num 8000 --eval_num 2000 --save_dir msmarco_data/10k

/home/hoang/DSI-QG/data
Creating MS MARCO dataset...
Reusing dataset ms_marco_passage_corpus (cache/Tevatron___ms_marco_passage_corpus/default/0.0.1/3d8add51914a7d2b589d09e37f4cd9646ebe4a3bad6d59b4e3702cfcc9941f23)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.79it/s]


In [5]:
import json

# Đường dẫn tới file JSONL
dev_path = "/home/hoang/DSI-QG/data/msmarco_data/10k/msmarco_DSI_dev_data.json"
train_path = "/home/hoang/DSI-QG/data/msmarco_data/10k/msmarco_DSI_train_data.json"

# Mở file và đọc từng dòng
with open(dev_path, 'r', encoding='utf-8') as file:
    dev_data = [json.loads(line) for line in file]
print("Number of example (all is query) in dev_data:", len(dev_data))

with open(train_path, 'r', encoding='utf-8') as file:
    train_data = [json.loads(line) for line in file]
print("Number of example in train_data:", len(train_data))
print("Number of document in train data:", (len(train_data) - len(dev_data)) // 2 + len(dev_data))
print("Number of query in train data:", (len(train_data) - len(dev_data)) // 2)

Number of example (all is query) in dev_data: 2000
Number of example in train_data: 18000
Number of document in train data: 10000
Number of query in train data: 8000


# Train Original DSI model

In [ ]:
%cd /home/hoang/DSI-QG
!CUDA_VISIBLE_DEVICES=0 python run.py \
        --task "DSI" \
        --model_name "google/mt5-base" \
        --run_name "msmarco-100k-mt5-base-DSI" \
        --max_length 256 \
        --train_file data/msmarco_data/100k/msmarco_DSI_train_data.json \
        --valid_file data/msmarco_data/100k/msmarco_DSI_dev_data.json \
        --output_dir "models/msmarco-100k-mt5-base-DSI" \
        --learning_rate 0.0005 \
        --warmup_steps 100 \
        --per_device_train_batch_size 8 \
        --per_device_eval_batch_size 8 \
        --evaluation_strategy steps \
        --eval_steps 1000 \
        --max_steps 2000 \
        --save_strategy steps \
        --dataloader_num_workers 10 \
        --save_steps 1000 \
        --save_total_limit 1 \
        --load_best_model_at_end \
        --gradient_accumulation_steps 1 \
        --report_to wandb \
        --logging_steps 100 \
        --dataloader_drop_last False \
        --metric_for_best_model Hits@10 \
        --greater_is_better True

/home/hoang/DSI-QG
/home/hoang/.conda/envs/dsi-qg/lib/python3.8/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/hoang/.conda/envs/dsi-qg/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Using custom data configuration default-b760cb11643ac61f
Reusing dataset json (cache/json/default-b760cb11643ac61f/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e65

# Train DSI-QG model

In [ ]:
!nvidia-smi --gpu-reset
import torch
torch.cuda.empty_cache()

## Step 1: Train a query generation model

In [ ]:
%cd /home/hoang/DSI-QG
!CUDA_VISIBLE_DEVICES=1 python run.py \
        --task "docTquery" \
        --model_name "google/mt5-large" \
        --run_name "docTquery-MSMARCO" \
        --max_length 128 \
        --train_file data/msmarco_data/100k/msmarco_DSI_train_data.json \
        --valid_file data/msmarco_data/100k/msmarco_DSI_dev_data.json \
        --output_dir "models/msmarco_docTquery_mt5_large" \
        --learning_rate 0.0001 \
        --warmup_steps 0 \
        --per_device_train_batch_size 4 \
        --per_device_eval_batch_size 4 \
        --evaluation_strategy steps \
        --eval_steps 100 \
        --max_steps 2000 \
        --save_strategy steps \
        --dataloader_num_workers 10 \
        --save_steps 100 \
        --save_total_limit 2 \
        --load_best_model_at_end \
        --gradient_accumulation_steps 4 \
        --report_to wandb \
        --logging_steps 100 \
        --dataloader_drop_last False

## Step 2: Run the query generation

In [ ]:
%cd /home/hoang/DSI-QG
!CUDA_VISIBLE_DEVICES=1 python run.py \
        --task generation \
        --model_name google/mt5-large \
        --model_path models/msmarco_docTquery_mt5_large/checkpoint-xxx \
        --per_device_eval_batch_size 32 \
        --run_name docTquery-MSMARCO-generation \
        --max_length 256 \
        --valid_file data/msmarco_data/100k/msmarco_corpus.tsv \
        --output_dir temp \
        --dataloader_num_workers 10 \
        --report_to wandb \
        --logging_steps 100 \
        --num_return_sequences 10

## Step 3: Train DSI-QG with query-represented corpus

In [ ]:
%cd /home/hoang/DSI-QG
!CUDA_VISIBLE_DEVICES=1 python run.py \
        --task "DSI" \
        --model_name "google/mt5-base" \
        --run_name "MSMARCO-100k-mt5-base-DSI-QG" \
        --max_length 32 \
        --train_file data/msmarco_data/100k/msmarco_corpus.tsv.q10.docTquery \
        --valid_file data/msmarco_data/100k/msmarco_DSI_dev_data.json \
        --output_dir "models/MSMARCO-100k-mt5-base-DSI-QG" \
        --learning_rate 0.0005 \
        --warmup_steps 100000 \
        --per_device_train_batch_size 32 \
        --per_device_eval_batch_size 32 \
        --evaluation_strategy steps \
        --eval_steps 1000 \
        --max_steps 1000000 \
        --save_strategy steps \
        --dataloader_num_workers 10 \
        --save_steps 1000 \
        --save_total_limit 2 \
        --load_best_model_at_end \
        --gradient_accumulation_steps 1 \
        --report_to wandb \
        --logging_steps 100 \
        --dataloader_drop_last False \
        --metric_for_best_model Hits@10 \
        --greater_is_better True \
        --remove_prompt True